In [1]:
import warnings
warnings.filterwarnings("ignore")
import sklearn
from sklearn.datasets import load_files
moviedir = r'./dataset/movie_reviews' 
movie_reviews = load_files(moviedir, shuffle=True)

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    movie_reviews.data, movie_reviews.target, test_size = 0.20, stratify=movie_reviews.target, random_state = 12)

In [3]:
from sklearn.svm import LinearSVC
clf = LinearSVC() 

IBM Watson

In [5]:
# Descomentá la siguiente linea para hacer la instalación
#!pip install watson-machine-learning-client --upgrade

In [6]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

2019-01-16 12:42:28,594 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.


Para usar Watson Machine Learning API es necesario crear un servicio desde:
https://console.bluemix.net/catalog/services/machine-learning.

Una vez creado ir a la lista de la izquierda a `Service Credentials` y crear nuevas credenciales. 

Para ver las credenciales hay que tocar en `View Credentials`. 
De allí copiar los datos necesarios para completar el código de la siguiente celda

In [4]:
wml_credentials={
  "url": "https://us-south.ml.cloud.ibm.com",
  "access_key": "vPoqnziiLJhJnqsnH2quLjpa0hG-zQx6-S_wRnH0wfdJ", # figura como apikey
  "username": "0ea59c9f-2daf-4568-80ec-62db24a243a1",
  "password": "f419d246-0775-4c2d-8397-654c60a5edc1",
  "instance_id": "a6a83106-6731-431f-b41b-1bfe43f20281"
}



In [7]:
client = WatsonMachineLearningAPIClient(wml_credentials)

Creamos los datos del modelo a publicar

In [8]:
model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "Fran", 
               client.repository.ModelMetaNames.AUTHOR_EMAIL: "fran@acamica.com",
               client.repository.ModelMetaNames.NAME: "Reviews classification"}

Creamos pipeline de entrenamiento

In [9]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
pipeline = make_pipeline(vect, clf)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_i...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

Subimos el modelo a la nube con `client.repository.store_model`


In [10]:
published_model = client.repository.store_model(model=pipeline, 
                                                meta_props=model_props, 
                                                training_data=X_train, 
                                                training_target=y_train)


Guardamos el uid del modelo

In [11]:
published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)

In [12]:
models_details = client.repository.list_models()

------------------------------------  ----------------------  ------------------------  -----------------
GUID                                  NAME                    CREATED                   FRAMEWORK
74b942f2-fa81-4bee-b766-bddbdf58fe61  Reviews classification  2019-01-16T15:42:29.518Z  scikit-learn-0.19
e382c633-9cf5-4432-89ed-ba4c8a267497  Reviews classification  2019-01-14T20:40:29.394Z  scikit-learn-0.19
------------------------------------  ----------------------  ------------------------  -----------------


Ahora podemos cargar el modelo a partir de su `uid`

In [13]:
loaded_model = client.repository.load(published_model_uid)
test_predictions = loaded_model.predict(X_test)

Mostramos el `classification_report`

In [14]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

print("roc_auc score: ", roc_auc_score(y_test, test_predictions))
print(classification_report(y_test, test_predictions))

roc_auc score:  0.85
             precision    recall  f1-score   support

          0       0.87      0.82      0.85       200
          1       0.83      0.88      0.85       200

avg / total       0.85      0.85      0.85       400

